# MODELO  NO SUPERVISADO 

In [20]:
# Se importan las librerías que se utilizarán: 
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import re
import array
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Se cargan los datasets de puntajes asginados por usuarios de las plataformas

#punt1 = pd.read_csv('..\\datasets\\puntajes\\1.csv')
#punt2 = pd.read_csv('..\\datasets\\puntajes\\2.csv')
#punt3 = pd.read_csv('..\\datasets\\puntajes\\3.csv')
punt4 = pd.read_csv('..\\datasets\\puntajes\\4.csv')
#punt5 = pd.read_csv('..\\datasets\\puntajes\\5.csv')
#punt6 = pd.read_csv('..\\datasets\\puntajes\\6.csv')
#punt7 = pd.read_csv('..\\datasets\\puntajes\\7.csv')
#punt8 = pd.read_csv('..\\datasets\\puntajes\\8.csv')

In [22]:
# Se borra la columna timestamp porque no se utilizará
punt4 = punt4.drop(columns = ['timestamp'])

In [23]:
# Se verifica el tipo de dato de cada columna
punt4. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   userId   1500000 non-null  int64  
 1   rating   1500000 non-null  float64
 2   movieId  1500000 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 34.3+ MB


In [24]:
# Se verifica que no hayan valores nulos
punt4.isna().sum()

userId     0
rating     0
movieId    0
dtype: int64

#### Se hace el filtrado de acuerdo a nuestro dataset para obtener las puntuaciones de sólamente las peliculas que se poseen

In [25]:
# Se importa el dataset final (CSV) y se visualiza
peliculas5=pd.read_csv(r'..\\datasets\\peliculas_final.csv')

el codigo de abajo no funciona


In [ ]:
# Se realiza el filtrado con la función merge
punt2 = pd.merge(punt4, peliculas5, left_on='movieId', right_on='show_id', how='inner')

# Se Seleccionan las columnas 'userId', 'rating' y 'movieId'
punt2 = punt4[['userId', 'rating', 'movieId']]

In [26]:
# Se realiza el filtrado con la función merge
punt2 = pd.merge(punt4, peliculas5, left_on='movieId', right_on='show_id', how='inner')

# Se Seleccionan las columnas 'userId', 'rating' y 'movieId'
punt2 = punt4[['userId', 'rating', 'movieId']]

In [28]:
# Se crea una columna plataforma con 
def asignar_plataforma(movieId):
    if 'a' in movieId:
        return 'Amazon'
    elif 'n' in movieId:
        return 'Netflix'
    elif 'd' in movieId:
        return 'Disney'
    elif 'h' in movieId:
        return 'hulu'
    
punt2['plataforma'] = punt2['movieId'].apply(asignar_plataforma)

In [29]:
punt2 = punt2[(punt2['plataforma'] != 'Netflix') & (punt2['plataforma'] != 'Disney')]

In [33]:
# Se realizan códigos para la columna plataforma que posee variables cualitativas
map_gender = {'Netflix': 1, 'Amazon': 2, 'Disney': 3}
punt2['plataforma'] = punt2['plataforma'].replace(map_gender)

In [34]:
# Se eliminan las letras de la columan movieId
def eliminar_letras(texto):
    return re.sub(r'[a-zA-Z]', '', texto)
# Se Aplica la función a la columna 'movieId'
punt2['movieId'] = punt2['movieId'].apply(eliminar_letras)

In [35]:
# Se guarda el dataset a csv
punt2.to_csv(r'..\datasets\punt2.csv', index=False)
punt2

,userId,rating,movieId,plataforma
0,46200,5.0,3203,2
1,46200,1.0,5959,2
2,46200,3.0,4430,2
3,46200,4.0,6604,2
5,46200,3.0,2438,hulu
...,...,...,...,...
1499995,61768,3.5,1021,hulu
1499996,61768,3.0,885,hulu
1499997,61768,4.0,4321,2
1499998,61768,4.0,2422,hulu


In [36]:
#Se escalan los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(punt2)
punt2_scaled = scaler.transform(punt2)
scaler = StandardScaler()
scaler.fit

ValueError: could not convert string to float: 'hulu'

In [ ]:
# Se observa la distribución
plt.figure(figsize=(20,8))
sns.countplot (x= "rating", data=punt2)
plt.title("Puntaje")
plt.xticks(rotation=90)
plt.show () 

In [ ]:
# Se realiza el método del codo para determinar la cantidad de clusters
k=np. arange(1,11)
lista_inercias = []
for i in k:
    kmeans = KMeans(n_clusters=i, random_state=0) 
# Instancia la clase KMeans
    kmeans.fit(x)  
# Ajusta el modelo KMeans a tus datos x
    lista_inercias.append(kmeans.inertia_)

In [ ]:
# Se grafica el método del codo
plt.figure(figsize=(7, 4))
plt.plot(k, lista_inercias, marker='o', linestyle='-', color='c')
plt.scatter(k, lista_inercias, c='r')
plt.xlabel('Cantidad de clusters (k)')
plt.ylabel('inercia media')
plt.title('The Elbow Method')
plt.show()

In [ ]:
# Se visualiza en otro tipo de gráfico
plt.plot(k, marker='o', linestyle='-', color='b')
plt.axvline(3, 0, 1, color='red')
plt.xlabel('Cantidad de clusters (k)')
plt.ylabel('Puntaje del modelo')
plt.title('Puntaje del modelo para determinar el número óptimo de clusters')
plt.show()

La cantidad de cluster a utilizar son 3

In [ ]:
# Se especifica la cantidad de clusters
num_clusters=3

# Se crea una instancia del modelo K-Means
kmeans = KMeans(n_clusters=num_clusters)

# Se ajusta el modelo a tus datos
kmeans.fit(x)
kmeans.fit(x)
kmeans
# Se obtiene las etiquetas de clúster para cada punto de datos
labels = kmeans.labels_

# Se obtiene las coordenadas de los centroides de los clústeres
centroids = kmeans.cluster_centers_
centroids = kmeans
print (centroids)

In [ ]:
# Se crea un gráfico de dispersión para visualizar la distribución de los clústeres
plt.figure(figsize=(8, 6))
sns.scatterplot(x='rating', y='userId', hue='cluster', data=x, palette='viridis')
plt.title('Distribución de Clústeres')
plt.show()

# Se importan las librerías que se utilizarán: 

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import re
import array
import warnings
warnings.filterwarnings('ignore')

# Se realiza el filtrado con la función merge
punt2 = pd.merge(punt1, peliculas5, left_on='movieId', right_on='show_id', how='inner')

# Se Seleccionan las columnas 'userId', 'rating' y 'movieId'
punt2 = punt2[['userId', 'rating', 'movieId']]

In [ ]:
# Se crea una columna plataforma con 
def asignar_plataforma(movieId):
    if 'a' in movieId:
        return 'Amazon'
    elif 'n' in movieId:
        return 'Netflix'
    elif 'd' in movieId:
        return 'Disney'
    elif 'h' in movieId:
        return 'hulu'
    
punt2['plataforma'] = punt2['movieId'].apply(asignar_plataforma)

In [ ]:
punt2 = punt2[(punt2['plataforma'] != 'Netflix') & (punt2['plataforma'] != 'Disney')]

# Se realizan códigos para la columna plataforma que posee variables cualitativas
map_gender = {'Netflix': 1, 'Amazon': 2, 'Disney': 3}
punt2['plataforma'] = punt2['plataforma'].replace(map_gender)

# Se eliminan las letras de la columan movieId
def eliminar_letras(texto):
    return re.sub(r'[a-zA-Z]', '', texto)
# Se Aplica la función a la columna 'movieId'
punt2['movieId'] = punt2['movieId'].apply(eliminar_letras)

# Se guarda el dataset a csv
punt2.to_csv(r'..\datasets\punt2.csv', index=False)
punt2

In [ ]:
#Se escalan los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(punt2)
punt2_scaled = scaler.transform(punt2)
scaler = StandardScaler()
scaler.fit

In [ ]:
# Se convierte la matriz numpy en un DataFrame
punt2_scaled_df = pd.DataFrame(punt2_scaled, columns=punt2.columns)
# Se seleccionan las variables
x = punt2_scaled_df.iloc[:, [1, 2]]

In [ ]:
# Se observa la distribución
plt.figure(figsize=(20,8))
sns.countplot (x= "rating", data=punt2)
plt.title("Puntaje")
plt.xticks(rotation=90)
plt.show () 

In [ ]:
# Se realiza el método del codo para determinar la cantidad de clusters
k=np. arange(1,11)
lista_inercias = []
for i in k:
    kmeans = KMeans(n_clusters=i, random_state=0) 
# Instancia la clase KMeans
    kmeans.fit(x)  
# Ajusta el modelo KMeans a tus datos x
    lista_inercias.append(kmeans.inertia_)

Se observa que la distribución de las muestras en 3 clusters no es igual en cada grupo. Se analizará posteriormente la distribución en dos clusters. 

In [ ]:
# Se observan las coordenandas de los centroides
centroids3 =kmeansmodel3.cluster_centers_
print(centroids3)

### N° de clusters:2

In [ ]:
# Se instancia el modelo con el numero de clusters 
kmeansmodel3 = KMeans(n_clusters=2, random_state=0)
kmeansmodel3.fit(p1_scal)


In [ ]:
# Etiquetas de los clusters
etiquetas_2 = kmeansmodel3.labels_
np.unique(etiquetas_2)

In [ ]:
# Se hace la predicción:
y_means2=kmeansmodel3.fit_predict(p1_scal)

In [ ]:
#Se verifica la cantidad de observaciones (2 clusters)
cluster_counts = np.bincount(y_means2)
cluster_labels = list(range(len(cluster_counts)))
colors = ['lightcoral', 'mediumturquoise', 'plum']
plt.bar(cluster_labels, cluster_counts, color=colors)
plt.title('Cantidad de muestras por cluster')
plt.xlabel('Grupo')
plt.ylabel('Cantidad de Observaciones')
plt.show()


Se puede observar que con dos clusters, la cantidad de muestras es bastante equitativa para cada grupo.

In [ ]:
# Se observan las coordenandas de los centroides
centroids2 =kmeansmodel2.cluster_centers_
print(centroids2)

In [ ]:
# Se verifica como performan
p1 = [x_subset]
df = pd.DataFrame(p1, columns=['X', 'Y'])
df['Cluster'] = etiquetas_1
sns.scatterplot(x='X', y='Y', hue='Cluster', p1=df, palette='pastel2')
plt.scatter(centroides[:, 0], centroides[:, 1], c='green', marker='x', s=200, label='Centroids1')
plt.title('Distribución de Clusters y Centroides')
plt.xlabel('Eje X')
plt.ylabel('Eje Y')
plt.legend()
plt.show()

In [ ]:
# Se evalua con la suma de los cuadrados de las distancias (SSD)
SSD= kmeansmodel2.inertia_
print( "SSD: ",SSD)

cuanto debe dar?


In [ ]:
# Se realiza la evaluación con silhoutte
sil = []
kmax = 10

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 3
for k in range(2, kmax+1):
    kmeans = KMeans(n_clusters=k).fit(p1_scal)
    labels = kmeans.labels_
    sil.append(silhouette_score(p1_scal, labels, metric='euclidean'))

In [ ]:
# Se grafica
plt.plot(range(2,11),sil, c="#c51b7d")
plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.title('Metodo Silhouette', size=14)
plt.xlabel('Numero de clusters', size=12)
plt.ylabel('Puntaje de Silhouette', size=14)
plt.show()

In [ ]:
# Se concatenan los datasets
#punt = pd.concat([punt1, punt2, punt3, punt4, punt5, punt6, punt7, punt8], axis=0, ignore_index=True)
#punt

Se debe considerar el coeficiente de silhoutte más cercano a 1, por lo que se reagruparán en 2 cluster. 